In [ ]:
import itertools
import numpy as np
import os
from tqdm import tqdm
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
import functools

%load_ext autoreload
%autoreload 2

from genpen.utils import Paper
import bezier
from dataclasses import asdict, dataclass, field

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
xstep = 25
ystep = 25
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=xstep, ystep=ystep, flatmesh=True)

In [ ]:
pts = []
for i, y in enumerate(ybins):
    for x in xbins:
        if (i % 2) == 0:
            x += xstep/4
        else:
            x += -xstep/4
        pts.append(Point(x,y))

In [ ]:
pts = MultiPoint(pts)
tris = so.triangulate(pts)
tris = [t for t in tris if t.intersects(drawbox.buffer(-21))]

In [ ]:
yrange = [drawbox.bounds[1]+40, drawbox.bounds[2]]

In [ ]:
jts = []
for t in tris:
    y = t.centroid.y
    angle_max = np.interp(y, yrange, [0, 10])
    angle = np.random.uniform(-angle_max, angle_max)
    jt = sa.rotate(t, angle)
    
    scale_max = np.interp(y, yrange, [0, 1.2])
    scale = np.random.uniform(1, 1+scale_max)
    jt = sa.scale(jt, xfact=scale, yfact=scale)
    
    jt = jt.buffer(-1, cap_style=2, join_style=1)
    jts.append(jt)

In [ ]:
boundaries = gp.merge_Polygons(jts).boundary

In [ ]:
bbs = so.unary_union([b.buffer(0.3, cap_style=1, join_style=1) for b in boundaries])

In [ ]:
bbbs = gp.merge_LineStrings([b.boundary for b in bbs if b.area > 1])
bbbs = gp.merge_Polygons([Polygon(b) for b in bbbs]).buffer(-0.1)
bbbs = so.unary_union([b.buffer(0.1, cap_style=1, join_style=1) for b in bbs.boundary])
bbbs = gp.merge_LineStrings([b.boundary for b in bbbs if b.area < 1e2])
bbbs = gp.merge_Polygons([Polygon(b).buffer(-0.1) for b in bbbs])

In [ ]:
bbbs = so.unary_union(bbbs)

In [ ]:
n_layers = 1
all_layers = [list() for i in range(n_layers)]
    
layer_choices = np.random.choice(n_layers, size=len(bbbs))
for i, lc in enumerate(layer_choices):
    p = bbbs[i]
    y = p.centroid.y
    angle_max = np.interp(y, yrange, [3, 50])
    angle = np.random.uniform(-angle_max, angle_max)
    
    spacing_range = np.interp(y, yrange, [0, 0.2])
    spacing = np.random.uniform(0.7-spacing_range, 0.7 + spacing_range)
    hatches = gp.hatchbox(p, spacing=spacing, angle=angle)
    all_layers[lc].append(hatches)
    
all_hatch_layers = []
for layer in all_layers:
    hatch_layer = [ls for ls in layer if ls.length > 1e-2]
    all_hatch_layers.append(gp.merge_LineStrings(hatch_layer))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
for i, layer in enumerate(all_hatch_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [1.2, 1.5]:
    sk.vpype(f'linesort linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify -t 0.2mm')
sk.vpype('linesort')
sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/231_randomness_y.svg')

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
xstep = 3
ystep = 3
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=xstep, ystep=ystep, flatmesh=True)

In [ ]:
pts = []
for i, y in enumerate(ybins):
    for x in xbins:
        if (i % 2) == 0:
            x += xstep/4
        else:
            x += -xstep/4
        std = np.interp(i, [20, len(ybins)], [0, 3]) ** 2
        coord = np.array([x,y]) + np.random.randn(2) * std
        pts.append(Point(*coord))

In [ ]:
pts = MultiPoint(pts)
tris = so.triangulate(pts)
tris = [t for t in tris if t.intersects(drawbox.buffer(-23))]

In [ ]:
gp.merge_Polygons(tris)

In [ ]:
jts = []
yrange = [drawbox.bounds[1]+180, drawbox.bounds[2]]
for t in tris:
    y = t.centroid.y
#     angle_max = np.interp(y, yrange, [0, 10])
#     angle = np.random.uniform(-angle_max, angle_max)
    jt = sa.rotate(t, 0)
    
    scale_max = np.interp(y, yrange, [0, 0.3])
    scale = np.random.uniform(1, 1+scale_max)
    jt = sa.scale(jt, xfact=scale, yfact=scale)
    
    jt = jt.buffer(-0.05, cap_style=2, join_style=2)
    jts.append(jt)

In [ ]:
morphed = gp.merge_Polygons(jts)

In [ ]:
n_layers = 1
all_layers = [list() for i in range(n_layers)]
yrange = [drawbox.bounds[1]+60, drawbox.bounds[2]]    
layer_choices = np.random.choice(n_layers, size=len(morphed))
for i, lc in enumerate(layer_choices):
    p = morphed[i]
    y = p.centroid.y
    angle_max = np.interp(y, yrange, [0, 13]) ** 2
    angle = np.random.uniform(-angle_max, angle_max)
    
    spacing_range = np.interp(y, yrange, [0, 0.4])
    spacing = np.random.uniform(0.65-spacing_range, 0.6 + spacing_range)
    hatches = gp.hatchbox(p, spacing=spacing, angle=angle)
    all_layers[lc].append(hatches)
    
all_hatch_layers = []
for layer in all_layers:
    hatch_layer = [ls for ls in layer if ls.length > 1e-2]
    all_hatch_layers.append(gp.merge_LineStrings(hatch_layer))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
for i, layer in enumerate(all_hatch_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.2, 0.5, 1, 2, 2]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm linesort ')
sk.vpype('linesimplify -t 0.1mm')
sk.vpype('linesort')
sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/236_randomness_y.svg')